In [3]:
import requests # text parsing
from bs4 import BeautifulSoup # text parsing
#from urllib.request import urlopen # reading files
import pandas as pd
import re
import nltk

nltk.download('vader_lexicon')
from nltk.sentiment.vader import SentimentIntensityAnalyzer

# catch HTTPError in url list
#from urllib.error import HTTPError

[nltk_data] Downloading package vader_lexicon to
[nltk_data]     /home/msoni/nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!


In [4]:
def parse(news_source, file, df):
    with open(file, "r") as f:
        for url in f:
            url = url.strip()

            res = requests.get(url)
            html_page = res.content
            soup = BeautifulSoup(html_page, 'html.parser')

            if news_source == "CNN":
                text = soup.find_all(class_ = ['zn-body__paragraph', 'el__storyhighlights__item', 'pg-headline'])
            elif news_source == "Breitbart":
                text = soup.find_all(['h1', 'p'])
            elif news_source == "NYT":
                text = soup.find_all(class_ = ['css-1vkm6nb ehdk2mb0', 'css-16f3y1r e13ogyst0', 'css-exrw3m evys1bk0'])

            all_text = ''
            for item in text:
                all_text += " " + item.get_text()
            temp_df = collect_sentiment(all_text, news_source)
            df = pd.concat([df, temp_df])
    
    return df

In [5]:
def collect_sentiment(all_text, news_source):
    analyser = SentimentIntensityAnalyzer()
    sentiment = analyser.polarity_scores(all_text)

    # variables
    hillary = ["hillary", "clinton", "Hillary", "Clinton"]
    trump = ["trump", "donald", "Donald", "Trump"]
    name = None
    word = "" # word to check if it is a name
    polarity_list = []
    name_list = []
    news_source_list = []
    sentence = ""
    
    # runs through the text and if name is found, adds sentiment to dataframe
    for item in all_text:
        # at end of sentence, gets sentiment
        if item == ".":
            sentiment = analyser.polarity_scores(sentence)
            sentence = ""
            continue

        # checks if word is a candidate name
        if item == " ":
            # removes all non-letter characters
            word = re.sub(r"[^\w\s_]+", "", word).strip()
            
            if word in hillary:
                word = "Hillary"
                polarity_list.append(sentiment)
                name_list.append(word)
                news_source_list.append(news_source)
            elif word in trump:
                word = "Trump"
                polarity_list.append(sentiment)
                name_list.append(word)
                news_source_list.append(news_source)
            word = ""

        # add a letter at a time
        sentence += item
        word += item

    # combine lists
    tuple_list = list(zip(name_list, polarity_list))
    
    # create dataframe of news source, candidate, and sentiment
    list_dict = {"News_Source": news_source_list, "Candidate": name_list, "Sentiment": polarity_list}
    df = pd.DataFrame(list_dict)
    return df

In [6]:
# gets all the files from different news sources
CNN_files = "Files/2016CNNArticleURLSBackup.txt"
Breitbart_files = "Files/2016BreitbartArticleURLS.txt"
NYT_files = "Files/2016NYTimesArticleURLS.txt"

df = pd.DataFrame()

df = parse("CNN", CNN_files, df)
df = parse("Breitbart", Breitbart_files, df)
df = parse("NYT", NYT_files, df)

In [8]:
df.to_csv(r"Dataset.csv", index = False)